# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. OpenAI 사이트에서 API 키 발급
2. google colab - secret 탭에 `OPENAI_API_KEY` 를 등록해야 한다.

In [1]:
from google.colab import userdata
from openai import OpenAI

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
print(response)

# ChatCompletion( choices=[ Choice( message=ChatCompletionMessage( content="여기가 실제 응답부분" ) ) ] )
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-Bls5Vd6xi3mA4NjSn2EaKxpVUd7cr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요, 차은우님! 반갑습니다. 어떻게 도와드릴까요?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750749029, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=19, prompt_tokens=41, total_tokens=60, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
안녕하세요, 차은우님! 반갑습니다. 어떻게 도와드릴까요?


## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

프롬프팅에 꼭 들어가있어야 할 내용들

1. Instruction ( 지시사항 )
2. Context ( 문맥. 정확히 어떤걸 해야 하는지 나열 )
3. Input Data/Example ( 입력데이터나 예시 )
4. Output Indicator ( 출력 지시사항. 어떤 형식으로 출력할지 )

**`매우 중요함!!`**

## 예제 1 : 기사 제목 교정
- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스 AFP 속보시스템에서 도입되어 사용한 적 있음.

In [ ]:
title_before = "테이의 FM 개꿀 라디오 방송에 주목해주세요. 꿀잠 쌉가능. 아 얼른 재취하고싶다."

# LLM 의 역할, 페르소나, 지시사항을 꼼꼼히 작성
system_message = """
기자들이 송고한 제목에서 맞춤법, 문법, 의미, 어조, 그 밖에 기사에 어울리지 않는 것들을 교정해주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하고 임팩트있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우, 표현을 완화하거나 중립적인 어조로 수정하세요.
- 주제에 맞지 않는 문구는 삭제해주세요.
- 비속어가 포함되어있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

# Step
1. 기사 제목을 읽고, 주요 내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 어필할 수 있는지 점검하고 간결하게 정리하세요.

# Output Format
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목 : [기사의 원래 제목]
- 교정 제목 : [교정된 기사 제목]

# Examples
- 원래 제목 : "어제 서울에서 졸라 불이 크게 나서 수백명이 대피했고, 꼰대들이 가장 큰 피해자였다."
- 교정 제목 : "서울에서 대형화제로 수백명 대피, 중장년층 피해 커"

# Extra instructions
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버리면 안됩니다.
- 지역명, 시간 등의 중요정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사 제목을 간결히 수정해주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response, "\n")
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-BlsUczmenCTSnkhbxXRTcJKSlvOvw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 원래 제목 : "테이의 FM 개꿀 라디오 방송에 주목해주세요. 꿀잠 쌉가능. 아 얼른 재취하고싶다."\n- 교정 제목 : "테이의 FM 라디오 방송, 편안한 수면을 위한 추천!"', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750750586, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=61, prompt_tokens=473, total_tokens=534, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))) 

- 원래 제목 : "테이의 FM 개꿀 라디오 방송에 주목해주세요. 꿀잠 쌉가능. 아 얼른 재취하고싶다."
- 교정 제목 : "테이의 FM 라디오 방송, 편안한 수면을 위한 추천!"


In [ ]:
# 함수 리팩토링
def audit_news_title(title, model="gpt-4o-mini", temperature=1.0, top_p=1.0):
  system_message = """
  기자들이 송고한 제목에서 맞춤법, 문법, 의미, 어조, 그 밖에 기사에 어울리지 않는 것들을 교정해주세요.

  - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
  - 독자의 관심을 끌 수 있도록 간결하고 임팩트있는 표현을 사용하세요.
  - 어조가 지나치게 감정적이거나 부정적인 경우, 표현을 완화하거나 중립적인 어조로 수정하세요.
  - 주제에 맞지 않는 문구는 삭제해주세요.
  - 비속어가 포함되어있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

  # Step
  1. 기사 제목을 읽고, 주요 내용을 이해하세요.
  2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
  3. 맞춤법, 문법, 의미 전달의 정확성을 점검하고 적절히 수정하세요.
  4. 제목이 자연스럽고, 독자에게 매력적으로 어필할 수 있는지 점검하고 간결하게 정리하세요.

  # Output Format
  기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

  - 원래 제목 : [기사의 원래 제목]
  - 교정 제목 : [교정된 기사 제목]

  # Examples
  - 원래 제목 : "어제 서울에서 졸라 불이 크게 나서 수백명이 대피했고, 꼰대들이 가장 큰 피해자였다."
  - 교정 제목 : "서울에서 대형화제로 수백명 대피, 중장년층 피해 커"

  # Extra instructions
  - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버리면 안됩니다.
  - 지역명, 시간 등의 중요정보는 명확하게 유지하세요.
  - 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

  """

  user_message = f"""
  다음 기사 제목을 간결히 수정해주세요.

  {title}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  print(response, "\n")

  return response.choices[0].message.content


title_before = "주말동안 천둥번개를 동반한 미친 폭우가 예상됩니다. 다들 쓸데없이 돌아다니지 말고 집에 처박혀있으세요."
print( audit_news_title(title_before) )

ChatCompletion(id='chatcmpl-BlshwoNuAWB69NrqZ4cKQj8UjZr44', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 원래 제목 : "주말동안 천둥번개를 동반한 미친 폭우가 예상됩니다. 다들 쓸데없이 돌아다니지 말고 집에 처박혀있으세요."\n- 교정 제목 : "주말 천둥번개 동반 폭우 예상, 외출 자제 권장"', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750751412, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=73, prompt_tokens=510, total_tokens=583, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))) 

- 원래 제목 : "주말동안 천둥번개를 동반한 미친 폭우가 예상됩니다. 다들 쓸데없이 돌아다니지 말고 집에 처박혀있으세요."
- 교정 제목 : "주말 천둥번개 동반 폭우 예상, 외출 자제 권장"


## 영문텍스트에서 anki 단어장 만들기 ( 과제 )

## 연애코치 ReAct 만들어보기
- 처음부터 함수형으로 만들어보자. 반복적으로 호출하기 쉽도록.

In [3]:
# 함수 리팩토링
def dating_coach(prompt, model="gpt-4o-mini", temperature=0.7, top_p=1.0):
  system_message = """
  <<< System Instruction >>>
  어떤 상황에서도 최고의 논리적/감성적 관점을 적용한 연애코치로서 사용자의 고민을 해결해주세요.

  <<< Output Format >>>
  1. 상황분석:

  2. 행동분석:

  3. 실행:

  """

  params = {
    "model"                 : model,
    "messages"              : [
                                { "role": "system",  "content": [{"type": "text", "text": system_message}] },
                                { "role": "user",    "content": [{"type": "text", "text": prompt}] }
                              ],
    "response_format"       : {"type": "text"},
    "temperature"           : temperature,
    "max_completion_tokens" : 2048,
    "top_p"                 : top_p,
    "frequency_penalty"     : 0,
    "presence_penalty"      : 0
  }

  response = client.chat.completions.create(**params)
  print(response, "\n")

  return response.choices[0].message.content


title_before = "새로 친해진 여자와 가까워지고 싶을 때 쓸 수 있는 대화주제"
print( dating_coach(title_before) )

ChatCompletion(id='chatcmpl-Bm905kEL75U3REaHqIE0PGs7G8NM8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. 상황분석:  \n새로 친해진 여자와 가까워지고 싶다는 것은 서로에 대한 관심이 있음을 나타내며, 관계를 발전시키고 싶은 마음이 담겨 있습니다. 하지만 처음에는 서로의 취향이나 성격을 잘 모르는 경우가 많기 때문에, 적절한 대화 주제를 선택하는 것이 중요합니다.\n\n2. 행동분석:  \n대화 주제를 선택할 때, 상대방의 관심사와 편안함을 고려해야 합니다. 너무 개인적인 질문은 부담을 줄 수 있고, 너무 일반적인 질문은 흥미를 잃게 할 수 있습니다. 따라서 자연스럽고 서로의 관심을 끌 수 있는 주제를 선택하는 것이 좋습니다.\n\n3. 실행:  \n다음과 같은 대화 주제를 활용해보세요:\n\n- **취미와 관심사**: "최근에 어떤 취미에 빠져있어?" 또는 "주말에 보통 뭐 하고 보내?" \n- **영화나 음악**: "최근에 본 영화 중에서 추천할 만한 거 있어?" 또는 "어떤 음악 장르를 좋아해?" \n- **여행**: "가장 기억에 남는 여행지는 어디야?" 또는 "가고 싶은 여행지가 있어?" \n- **음식**: "좋아하는 음식이나 레스토랑이 있어?" 또는 "요리하는 거 좋아해?" \n- **일상적인 이야기**: "최근에 재미있는 일이 있었어?" 또는 "일상에서 스트레스 푸는 방법이 있어?" \n\n이런 주제들은 자연스럽게 대화를 이어갈 수 있는 기회를 제공하며, 서로의 생각과 감정을 나누는 데 도움을 줄 것입니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750814041, model='gpt-4o-mini-202

## 냉털 마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해주게 만든다.
- Reasoning/Action 을 끌어낼 수 있는 적절한 프롬프팅을 작성해본다.

In [4]:
from IPython.display import Markdown

def fridge_raid_master(prompt, model="gpt-4o-mini", temperature=0.7, top_p=1.0):
  system_message = """
  냉장고에 남은 재료와 반찬들을 사용해 창의적이고 맛있는 요리 아이디어를 제시하세요.

  - 사용 가능한 재료 목록을 확인합니다.
  - 각 재료의 특성과 조화를 고려하여 요리 아이디어를 제안합니다.
  - 요리 과정의 주요 단계를 설명하여 요리를 만들기 쉽게 안내합니다.
  - 조리시간이 길지 않은 요리 아이디어를 제안합니다.

  # Steps

  1. 사용 가능한 재료 목록을 작성합니다.
  2. 각 재료의 맛과 조합 가능성을 토대로 요리 가능성을 평가합니다.
  3. 가장 적합한 요리 아이디어를 하나 이상 제안합니다.
  4. 각 요리의 조리법과 필요한 단계를 설명합니다.

  # Output Format

  간결한 단락 형태로 재료에 맞는 요리 아이디어와 단계별 조리법 설명을 제시하세요.

  # Examples

  **Input:**
  - 재료: 닭고기, 당근, 양파, 감자, 고추장

  **Output:**
  - 요리 아이디어: 매콤한 닭고기 찜
  - 조리법:
    1. 닭고기를 적당한 크기로 자른 후 소금과 후추로 간을 합니다.
    2. 양파는 얇게 자르고, 당근과 감자는 먹기 좋은 크기로 자릅니다.
    3. 팬에 기름을 두르고 양파를 넣어 볶다, 닭고기를 넣고 겉면이 익을 정도로 볶습니다.
    4. 고추장을 넣어 매콤한 맛을 더한 후, 당근과 감자를 넣어 센 불에서 볶습니다.
    5. 약한 불에서 뚜껑을 덮고 찌며 재료가 완전히 익을 때까지 조리합니다.

  (**실제 예제는 재료 목록이 다를 수 있음**)

  # Notes

  - 가능한 다양한 재료 조합을 고려하여 다채로운 요리를 제안하세요.
  - 특별한 조리 기구 또는 복잡한 조리법은 피하고, 쉽게 구현할 수 있는 방법을 추천합니다.

  """

  params = {
    "model"                 : model,
    "messages"              : [
                                { "role": "system",  "content": [{"type": "text", "text": system_message}] },
                                { "role": "user",    "content": [{"type": "text", "text": prompt}] }
                              ],
    "response_format"       : {"type": "text"},
    "temperature"           : temperature,
    "max_completion_tokens" : 2048,
    "top_p"                 : top_p,
    "frequency_penalty"     : 0,
    "presence_penalty"      : 0
  }

  response = client.chat.completions.create(**params)
  print(response, "\n")

  return response.choices[0].message.content


title_before = "호박볶음, 파, 생선, 계란, 무말랭이, 김치, 배추간장절임"
display( Markdown( fridge_raid_master(title_before) ) )

ChatCompletion(id='chatcmpl-Bm99Uvog48NlaE2JJyngVKEikdyyv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 요리 아이디어: 생선 계란 볶음밥\n- 조리법:\n  1. 생선을 적당한 크기로 썰고 팬에 기름을 두른 후 중불에서 살짝 볶아줍니다. 생선이 익을 때까지 볶아주세요.\n  2. 파는 송송 썰고, 호박볶음은 미리 조리된 것을 준비합니다.\n  3. 생선이 익으면 파와 호박볶음을 팬에 넣고 함께 볶아줍니다.\n  4. 계란을 깨서 팬에 넣고 재료들과 잘 섞이도록 저어가며 볶습니다.\n  5. 마지막으로 간을 맞추고, 원한다면 무말랭이와 김치를 곁들여 함께 서빙합니다. \n\n이 요리는 15분 정도 걸리며, 남은 재료를 활용하여 쉽게 만들 수 있습니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750814624, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=207, prompt_tokens=542, total_tokens=749, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, 

## 면접 질문지를 자동으로 만들어주는 챗봇 만들어보기 JSON 출력

In [6]:
from IPython.display import Markdown

def job_interview(prompt, model="gpt-4o-mini", temperature=0.7, top_p=1.0):
  system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로서 해당분야 최고의 면접관입니다.
매번 인재를 발굴하기 위해 면접질문/모범답안을 작성하고있습니다.

<< 지시사항 >>
- 사용자가 제출한 채용공고 내용을 바탕으로 예상 면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두 개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 세 개 씩 만들어주세요.

<< 출력형식 >>
출력은 json 형식으로만 반환해주세요.

{{
  "hard_skill" : [
    {{
      "질문" : "", "답변"
    }}
  ],
  "soft_skill" : [
    {{
      "질문" : "", "답변"
    }}
  ]
}}
  """
  user_message = f"""
채용공고 :
{prompt}
  """

  params = {
    "model"                 : model,
    "messages"              : [
                                { "role": "system",  "content": [{"type": "text", "text": system_message}] },
                                { "role": "user",    "content": [{"type": "text", "text": prompt}] }
                              ],
    "response_format"       : {"type": "json_object"},
    "temperature"           : temperature,
    "max_completion_tokens" : 2048,
    "top_p"                 : top_p,
    "frequency_penalty"     : 0,
    "presence_penalty"      : 0
  }

  response = client.chat.completions.create(**params)
  print(response, "\n")

  return response.choices[0].message.content

In [8]:
prompt = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>   (MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |

"""
output = job_interview(prompt)
print( output )

ChatCompletion(id='chatcmpl-BmA6c7orOyz3TyvotY4x7N0xa1dda', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "hard_skill": [\n    {\n      "질문": "Python을 사용하여 머신러닝 인프라를 구축한 경험에 대해 설명해 주시겠습니까?",\n      "답변": "저는 지난 프로젝트에서 Python을 사용하여 머신러닝 모델을 위한 배포 인프라를 구축했습니다. Flask를 이용하여 REST API를 만들고, Docker를 활용해 이를 컨테이너화했습니다. Kubernetes를 통해 여러 컨테이너를 관리하며, Jenkins를 이용한 CI/CD 파이프라인을 구축해 지속적인 통합과 배포를 자동화했습니다."\n    },\n    {\n      "질문": "컨테이너 오케스트레이션 툴인 Kubernetes의 주요 기능과 이를 활용한 경험을 말씀해 주시겠습니까?",\n      "답변": "Kubernetes는 자동화된 컨테이너 배포, 스케일링 및 관리를 제공하는 플랫폼입니다. 저는 Kubernetes를 사용하여 머신러닝 모델의 배포와 관리를 수행했습니다. 예를 들어, 여러 버전의 모델을 동시에 운영하고, 필요에 따라 리소스를 자동으로 조정하는 기능을 활용하여 시스템의 안정성을 높였습니다."\n    },\n    {\n      "질문": "MLflow를 사용한 프로젝트 경험에 대해 설명해 주시겠습니까?",\n      "답변": "MLflow를 사용하여 모델의 실험 관리와 배포를 수행한 경험이 있습니다. 다양한 하이퍼파라미터 조합으로 실험을 진행하고, 각 실험 결과를 MLflow의 UI를 통해 비교했습니다. 최종적으로 성능이 가장 뛰어난 모델을 선정하여, MLflow의 모델 서빙 기능을 통해 프로덕션 환경에 배포했습니다."\n    }\n  ],\n  "soft_

In [9]:
import json

data = json.loads(output)
print(type(data))

hard_skill_qa = data["hard_skill"]
soft_skill_qa = data["soft_skill"]
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'질문': 'Python을 사용하여 머신러닝 인프라를 구축한 경험에 대해 설명해 주시겠습니까?', '답변': '저는 지난 프로젝트에서 Python을 사용하여 머신러닝 모델을 위한 배포 인프라를 구축했습니다. Flask를 이용하여 REST API를 만들고, Docker를 활용해 이를 컨테이너화했습니다. Kubernetes를 통해 여러 컨테이너를 관리하며, Jenkins를 이용한 CI/CD 파이프라인을 구축해 지속적인 통합과 배포를 자동화했습니다.'}, {'질문': '컨테이너 오케스트레이션 툴인 Kubernetes의 주요 기능과 이를 활용한 경험을 말씀해 주시겠습니까?', '답변': 'Kubernetes는 자동화된 컨테이너 배포, 스케일링 및 관리를 제공하는 플랫폼입니다. 저는 Kubernetes를 사용하여 머신러닝 모델의 배포와 관리를 수행했습니다. 예를 들어, 여러 버전의 모델을 동시에 운영하고, 필요에 따라 리소스를 자동으로 조정하는 기능을 활용하여 시스템의 안정성을 높였습니다.'}, {'질문': 'MLflow를 사용한 프로젝트 경험에 대해 설명해 주시겠습니까?', '답변': 'MLflow를 사용하여 모델의 실험 관리와 배포를 수행한 경험이 있습니다. 다양한 하이퍼파라미터 조합으로 실험을 진행하고, 각 실험 결과를 MLflow의 UI를 통해 비교했습니다. 최종적으로 성능이 가장 뛰어난 모델을 선정하여, MLflow의 모델 서빙 기능을 통해 프로덕션 환경에 배포했습니다.'}]
[{'질문': '팀 내에서 발생한 갈등을 어떻게 해결했는지에 대한 경험을 이야기해 주시겠습니까?', '답변': '이전 프로젝트에서 팀원 간의 의견 차이로 갈등이 발생했습니다. 저는 중재 역할을 맡아 각 팀원의 의견을 경청하고, 공통의 목표를 다시 한 번 강조했습니다. 이를 통해 서로의 입장을 이해하고, 최선의 해결책을 찾는 데 집중하여 갈등을 해결할 수 있었습니다.'}, {'질문': '기술 지원을 제공할 때 가장 중요하게 생각하는 점은 무엇인가요?', '답변